In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk


In [2]:
#file_path = 'C:\Users\Admin\Downloads\archivenow\datsetforchatbot.csv'
file_path = 'C:\\Users\\Admin\\Downloads\\archivenow\\datsetforchatbot.csv'


dataset = pd.read_csv(file_path)


In [3]:
print(dataset.head())
print(dataset.info())


   flags                                          utterance category  \
0   BILC  I don't have an online account, what do I have...  ACCOUNT   
1  BILQZ  can you tell me if i can regisger two accounts...  ACCOUNT   
2   BPLC         I have no online account, open one, please  ACCOUNT   
3  BIPLD  could you ask an agent how to open an account,...  ACCOUNT   
4   BLQC               i want an online account, create one  ACCOUNT   

           intent  
0  create_account  
1  create_account  
2  create_account  
3  create_account  
4  create_account  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21534 entries, 0 to 21533
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   flags      21534 non-null  object
 1   utterance  21534 non-null  object
 2   category   21534 non-null  object
 3   intent     21534 non-null  object
dtypes: object(4)
memory usage: 673.1+ KB
None


In [4]:
unique_intents = dataset['intent'].unique()
print(unique_intents)


['create_account' 'delete_account' 'edit_account' 'recover_password'
 'registration_problems' 'switch_account' 'check_cancellation_fee'
 'contact_customer_service' 'contact_human_agent' 'delivery_options'
 'delivery_period' 'complaint' 'review' 'check_invoices' 'get_invoice'
 'newsletter_subscription' 'cancel_order' 'change_order' 'place_order'
 'track_order' 'check_payment_methods' 'payment_issue'
 'check_refund_policy' 'get_refund' 'track_refund'
 'change_shipping_address' 'set_up_shipping_address']


In [5]:
from nltk.tokenize import word_tokenize

# Tokenize the 'utterance' column and create a new column 'tokens' to store the tokens
dataset['tokens'] = dataset['utterance'].apply(word_tokenize)

# Convert tokens to lowercase
dataset['tokens'] = dataset['tokens'].apply(lambda x: [token.lower() for token in x])



In [6]:
from sklearn.model_selection import train_test_split

# Split the dataset into features (X) and target (y)
X = dataset['tokens']  # Features (input data)
y = dataset['intent']  # Target (output labels)

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the training and testing sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)


Training set shape: (17227,) (17227,)
Testing set shape: (4307,) (4307,)


In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training text data to TF-IDF vectors
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train.apply(lambda x: ' '.join(x)))

# Initialize and train a Multinomial Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)


MultinomialNB()

In [8]:
# Transform the testing text data to TF-IDF vectors
X_test_tfidf = tfidf_vectorizer.transform(X_test.apply(lambda x: ' '.join(x)))

# Evaluate the model's accuracy on the testing set
accuracy = model.score(X_test_tfidf, y_test)
print("Model Accuracy:", accuracy)


Model Accuracy: 0.964476433712561


In [9]:
# Example user input
user_input = "I want to cancel my order"

# Preprocess the user input (tokenization and lowercase conversion)
user_tokens = word_tokenize(user_input.lower())

# Convert user tokens to a TF-IDF vector using the trained TF-IDF vectorizer
user_input_tfidf = tfidf_vectorizer.transform([' '.join(user_tokens)])

# Predict the intent of the user query using the trained model
predicted_intent = model.predict(user_input_tfidf)[0]

print("Predicted Intent:", predicted_intent)


Predicted Intent: delete_account


In [10]:
import pickle


In [11]:
import pickle

# Save the trained model to a pickle file
with open("model.pkl", "wb") as file:
    pickle.dump(model, file)


In [12]:
import pickle

# Save the TF-IDF vectorizer to a pickle file
with open("tfidf_vectorizer.pkl", "wb") as file:
    pickle.dump(tfidf_vectorizer, file)


In [13]:
# Load trained model
with open("model.pkl", "rb") as file:
    model = pickle.load(file)

# Load TF-IDF vectorizer
with open("tfidf_vectorizer.pkl", "rb") as file:
    tfidf_vectorizer = pickle.load(file)


In [14]:
import tkinter as tk
from tkinter import scrolledtext


In [15]:
import nltk
from nltk.tokenize import word_tokenize

# Create a Tkinter window
window = tk.Tk()
window.title("Customer Service Chatbot")

# Create a scrolled text widget to display the chat history
chat_history = scrolledtext.ScrolledText(window, width=60, height=20)
chat_history.grid(row=0, column=0, padx=10, pady=10)

# Create an entry widget for user input
user_input = tk.Entry(window, width=50)
user_input.grid(row=1, column=0, padx=10, pady=10)

def get_response(intent):
    responses = {
        "create_account": "I can help you create a new account. Please provide me with the necessary details.",
        "delete_account": "To delete your account, please contact our customer service team for assistance.",
        "edit_account": "You can edit your account details by logging into your account and accessing the settings.",
        "recover_password": "To recover your password, please use the 'Forgot Password' option on the login page.",
        "registration_problems": "If you're facing issues with registration, please ensure all required fields are filled correctly.",
        "switch_account": "You can switch between accounts by logging out and logging in with a different account.",
        "check_cancellation_fee": "To check cancellation fees, please refer to our terms and conditions or contact customer service.",
        "contact_customer_service": "Please feel free to contact our customer service team at support@example.com or call us at +1-800-123-4567 for assistance.",
        "contact_human_agent": "To speak with a human agent, please contact our customer service team directly.",
        "delivery_options": "We offer standard and express delivery options. Standard delivery typically takes 3-5 business days, while express delivery arrives within 1-2 business days.",
        "delivery_period": "The delivery period depends on your location and chosen delivery option. Please check our website for estimated delivery times.",
        "complaint": "We apologize for any inconvenience caused. Please provide more details about your complaint so we can assist you further.",
        "review": "Thank you for your review! We appreciate your feedback and use it to improve our services.",
        "check_invoices": "You can check your invoices by logging into your account and accessing the invoices section.",
        "get_invoice": "To get a specific invoice, please log into your account and download it from the invoices section.",
        "newsletter_subscription": "To subscribe to our newsletter, please visit our website and enter your email address in the subscription form.",
        "cancel_order": "Your order has been successfully cancelled.",
        "change_order": "To change your order, please contact customer service as soon as possible.",
        "place_order": "You can place a new order through our website or mobile app.",
        "track_order": "To track your order, please enter your order number on the order tracking page.",
        "check_payment_methods": "You can check available payment methods during the checkout process on our website.",
        "payment_issue": "If you're experiencing payment issues, please ensure that your payment information is entered correctly. You can also try using a different payment method or contacting your bank for assistance.",
        "check_refund_policy": "Our refund policy details can be found on our website under the terms and conditions section.",
        "get_refund": "To request a refund, please contact customer service with your order details.",
        "track_refund": "To track the status of your refund, please contact customer service.",
        "change_shipping_address": "To change your shipping address, please log into your account and update your address details.",
        "set_up_shipping_address": "To set up a new shipping address, please log into your account and add the address in the shipping settings."
    }
    
    # Return the response corresponding to the predicted intent
    return responses.get(intent, "I'm sorry, I'm not sure how to respond to that.")



# Function to handle user input and display chatbot response
def send_message():
    # Get user input
    user_query = user_input.get()
    
    # Preprocess user input (tokenization and lowercase conversion)
    user_tokens = word_tokenize(user_query.lower())

    # Convert user tokens to a TF-IDF vector using the trained TF-IDF vectorizer
    user_input_tfidf = tfidf_vectorizer.transform([' '.join(user_tokens)])

    # Predict the intent of the user query using the trained model
    predicted_intent = model.predict(user_input_tfidf)[0]
    
    # Get the appropriate response for the predicted intent (you can customize this based on your intents)
    response = get_response(predicted_intent)
    
    # Display user input and chatbot response in the chat history
    chat_history.insert(tk.END, "You: " + user_query + "\n")
    chat_history.insert(tk.END, "Chatbot: " + response + "\n")
    
    # Clear the user input entry widget
    user_input.delete(0, tk.END)


# Create a button to send user input
send_button = tk.Button(window, text="Send", command=send_message)
send_button.grid(row=1, column=1, padx=10, pady=10)

# Run the Tkinter event loop
window.mainloop()